In [1]:
# inporting our own functions
from modules.TrainTestValGen import init_gens
# @TODO add more to the modules folder to save space 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from _thread import start_new_thread

import tensorflow as tf
from keras import backend as K
from keras import regularizers, optimizers

from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization

from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session

In [3]:
import time
from datetime import timedelta

import math
import os

import scipy.misc
from scipy.stats import itemfreq
from random import sample
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Image manipulation
import PIL.Image
from IPython.display import display

In [4]:
# Load data
BBox_df = pd.read_csv('../BBox_List_2017.csv')
de_df = pd.read_csv('../Data_Entry_2017.csv')
train_filter = pd.read_csv('../train_val_list.txt')
train_filter.columns=['Image_Index']
test_filter = pd.read_csv('../test_list.txt')
test_filter.columns=['Image_Index']

In [5]:
# clean up incorrect ages in the data
bad_ages = de_df[de_df['Patient Age'] >= 100].index

df = de_df.drop(bad_ages)

df.drop(columns=['Unnamed: 11','OriginalImagePixelSpacing[x','y]','OriginalImage[Width','Height]'], inplace=True)

df.columns = ['Image_Index', 'Finding_Labels', 'Follow-up', 'Patient ID',
       'Patient_Age', 'Patient_Gender', 'View_Position']

# Remove secondary labels
# @TODO handle labels better
df['Finding_Labels'] = df['Finding_Labels'].apply(lambda x: x.split('|')[0])


# Join to the list of test and training to split the data set
train_df = pd.merge(train_filter,df , on='Image_Index', how='inner')
test_df = pd.merge(test_filter,df , on='Image_Index', how='inner')

print(f'train_df =  {len(train_df)}  test_df =  {len(test_df)}')

train_df =  86511  test_df =  25591


In [6]:
#Size Images 
# @TODO test to see what the pixle size is 
datagen = ImageDataGenerator(rescale=.125,validation_split=0.25)
test_datagen=ImageDataGenerator(rescale=.125)

# How we bin out the batches to run onto the GPU
BATCH_SIZE = 20

In [6]:
# @TODO Get this working an import


#Attach CSV to Images Folder and defline our traiing validation and test
def vaild_gen(batch_fit):
    global valid_generator
    valid_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image_Index",
        y_col="Finding_Labels",
        subset="validation",
        class_mode="categorical",
        target_size=(128,128),
        batch_size=batch_fit)
    
def train_gen(batch_fit):
    global train_generator
    train_generator=datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image_Index",
        y_col="Finding_Labels",
        subset="training",
        class_mode="categorical",
        target_size=(128,128),
        batch_size=batch_fit)
    
def test_gen(batch_fit):
    global test_generator
    test_generator=test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory="../images/",
        x_col="Image_Index",
        y_col="Finding_Labels",
        class_mode="categorical",
        target_size=(128,128),
        batch_size=batch_fit)
    
def init_gens(batch_fit):
    start_new_thread(vaild_gen,(batch_fit, ))
    start_new_thread(train_gen,(batch_fit ,))
    start_new_thread(test_gen,(batch_fit, ))
    
# Resetting global Memory https://github.com/keras-team/keras/issues/12625
# TODO move to module
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass
    # use the same config as you used to create the session
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.Session(config=config))

In [9]:
init_gens(BATCH_SIZE)

Found 23438 validated image filenames belonging to 15 classes.
Found 19666 validated image filenames belonging to 15 classes.
Found 59001 validated image filenames belonging to 15 classes.


In [11]:
# @TODO read https://medium.com/nanonets/how-to-easily-build-a-dog-breed-image-classification-model-2fd214419cde
# and build a better CNN
#layers
model = Sequential()
model.add(Conv2D(64, (7, 7), padding='same',
                 input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (7, 7)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (7, 7), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (7, 7)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(15, activation='softmax'))

In [12]:
#test Shape of data
train_labels = to_categorical(train_df.shape)
train_labels.shape

(2, 86512)

In [13]:
STEP_SIZE_VALID= valid_generator.n//valid_generator.batch_size

STEP_SIZE_TRAIN= train_generator.n//train_generator.batch_size

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


model.compile(optimizers.rmsprop(lr=0.0001),
loss="categorical_crossentropy", metrics=["accuracy"])


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    workers=16,
                    use_multiprocessing=False,
                    epochs=15) 

W0729 03:28:37.767798 14744 deprecation_wrapper.py:119] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0729 03:28:37.771799 14744 deprecation_wrapper.py:119] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0729 03:28:37.825811 14744 deprecation.py:323] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
2950/2950 [==============================] - 210s 71ms/step - loss: 6.9882 - acc: 0.5662 - val_loss: 6.0324 - val_acc: 0.6257
Epoch 2/15
2950/2950 [==============================] - 203s 69ms/step - loss: 6.9928 - acc: 0.5662 - val_loss: 6.0318 - val_acc: 0.6258
Epoch 3/15
2950/2950 [==============================] - 203s 69ms/step - loss: 6.9868 - acc: 0.5665 - val_loss: 6.0326 - val_acc: 0.6257
Epoch 4/15
2950/2950 [==============================] - 204s 69ms/step - loss: 6.9914 - acc: 0.5662 - val_loss: 6.0326 - val_acc: 0.6257
Epoch 5/15
2950/2950 [==============================] - 204s 69ms/step - loss: 6.9914 - acc: 0.5662 - val_loss: 6.0285 - val_acc: 0.6260
Epoch 6/15
2950/2950 [==============================] - 202s 68ms/step - loss: 6.9884 - acc: 0.5664 - val_loss: 6.0351 - val_acc: 0.6256
Epoch 7/15
2950/2950 [==============================] - 202s 68ms/step - loss: 6.9898 - acc: 0.5663 - val_loss: 6.0318 - val_acc: 0.6258
Epoch 8/15
2950/2950 [===================